<h2>Lab 7: Compression</h2>

Mehak Sharma  |  BioE 131  |  Fall 2019

<b>Simulating the data</b>

In [45]:
import numpy as np
import pandas as pd

In [46]:
def data_gen(size, percentageZeros):
    data = np.random.choice([0,1], size=size, replace=True, p=[percentageZeros, 1.0-percentageZeros])
    data = np.packbits(data)
    return data

def DNA_gen(size):
    data = np.random.choice(['A','T','C','G'], size=size, replace=True, p=[0.25 for _ in range(4)])
    return data
    
def protein_gen(size):
    data = np.random.choice(list('ARNDCEQGHILKMFPSTWYV'), size=size, replace=True, p=[0.05 for _ in range(20)])
    return data
    
open('zeros_100p', 'wb').write(data_gen(8 * 10**6, 1.0))
open('zeros_90p', 'wb').write(data_gen(8 * 10**6, 0.9))
open('zeros_80p', 'wb').write(data_gen(8 * 10**6, 0.8))
open('zeros_70p', 'wb').write(data_gen(8 * 10**6, 0.7))
open('zeros_60p', 'wb').write(data_gen(8 * 10**6, 0.6))
open('zeros_50p', 'wb').write(data_gen(8 * 10**6, 0.5))

open('nt_seq.fa', 'w').write(''.join(DNA_gen(10**6))) 
open('aa_seq.fa', 'w').write(''.join(protein_gen(10**6)))

1000000

I ran commands similar to these on my terminal:
- time gzip –c zeros_100p > zeros_100p.gz
- time bzip2 –k zeros_100p 
- time pbzip2 –k zeros_100p 

In [47]:
d = {'zeros_100p': ["105MB, 102 kB, 0.717s", "105MB, 113B, 1.020s", "105MB, 5.62KB, 0.104s"],
     'zeros_90p' : ["105MB, 58.7MB, 18.49s", "105MB, 61.7MB, 10.670s", "105MB, 61.2MB, 0.777s"],
     'zeros_80p' : ["105MB, 81.2MB, 13.407s", "105MB, 86.6MB, 11.941s", "105MB, 86.7MB, 0.981s"],
     'zeros_70p' : ["105MB, 93.6MB, 6.026s", "105MB, 99.8MB, 13.769s", "105MB, 99.8MB, 1.175s"],
     'zeros_60p' : ["105MB, 102MB, 4.373s", "105MB, 105MB, 15.721s", "105MB, 105MB, 1.433s"],
     'zeros_50p' : ["105MB, 102MB, 3.655s", "105MB, 105MB, 16.730s", "105MB, 105MB, 1.451s"],
    }
dF = pd.DataFrame(data=d, index=['gzip', 'bzip2', 'pbzip2'])
dF

,zeros_100p,zeros_90p,zeros_80p,zeros_70p,zeros_60p,zeros_50p
gzip,"105MB, 102 kB, 0.717s","105MB, 58.7MB, 18.49s","105MB, 81.2MB, 13.407s","105MB, 93.6MB, 6.026s","105MB, 102MB, 4.373s","105MB, 102MB, 3.655s"
bzip2,"105MB, 113B, 1.020s","105MB, 61.7MB, 10.670s","105MB, 86.6MB, 11.941s","105MB, 99.8MB, 13.769s","105MB, 105MB, 15.721s","105MB, 105MB, 16.730s"
pbzip2,"105MB, 5.62KB, 0.104s","105MB, 61.2MB, 0.777s","105MB, 86.7MB, 0.981s","105MB, 99.8MB, 1.175s","105MB, 105MB, 1.433s","105MB, 105MB, 1.451s"


In [48]:
d1 = {'seq0': ["100MB, 29.2MB, 12.140s", "100MB, 27.3MB, 9.487s", "100MB, 27.3MB, 0.682s"],
     'seq1' : ["100MB, 29.2MB, 12.154s", "100MB, 27.3MB, 10.132s", "100MB, 27.3MB, 0.671s"],
     'seq2' : ["100MB, 29.2MB, 12.142s", "100MB, 27.3MB, 9.650s", "100MB, 27.3MB, 0.674s"],
    }
dF2 = pd.DataFrame(data=d1, index=['gzip', 'bzip2', 'pbzip2'])
dF2

,seq0,seq1,seq2
gzip,"100MB, 29.2MB, 12.140s","100MB, 29.2MB, 12.154s","100MB, 29.2MB, 12.142s"
bzip2,"100MB, 27.3MB, 9.487s","100MB, 27.3MB, 10.132s","100MB, 27.3MB, 9.650s"
pbzip2,"100MB, 27.3MB, 0.682s","100MB, 27.3MB, 0.671s","100MB, 27.3MB, 0.674s"


- Which algorithm achieves the best level of compression on each file type? <b>From zero percentage 100% to 50%, the best algorithms are bzip2, bzip2, bzip2, bzip2, gzip, gzip. For the DNA sequences, the best compression algorithm is tied between bzip2 and pbzip2.</b>

- Which algorithm is the fastest? <b>bzip2 is the fastest algorithm.</b>

- What is the difference between bzip2 and pbzip2? Do you expect one to be faster and why? <b>pbzip2 is significantly faster than bzip2. It appears that bzip2 is faster at compressing files with a higher percentage of zeroes. In general, pbzip is likely faster since it runs in parallel.</b>

- How does the level of compression change as the percentage of zeros increases? Why does this happen? <b>The compression of the file appears to be inversely related to the percentage of zeroes, probably since there are fewer sections of uninterrupted entries of the same kind (zeros) that can be compressed.</b>

- What is the minimum number of bits required to store a single DNA base? <b>0.25 bits because 25MB are sufficient to encode 100MB of DNA data.</b>

- What is the minimum number of bits required to store an amino acid letter? <b>4.322 </b>

- In your tests, how many bits did gzip and bzip2 actually require to store your random DNA and protein sequences? <b>gzip and bzip2 took 29.2 and 27.3 MB respectively.</b>

- Are gzip and bzip2 performing well on DNA and proteins? <b>gzip and bzip2 are both only a few MB off from the ideal 25MB.</b>


<b>Compressing real data</b>

Now that you have a sense of how random data can be compressed, let’s have a look at some real biological data. Using what you’ve learned about querying biological databases, find the nucleic acid sequences of gp120 homologs from at least 10 different HIV isolates and concatenate them together into a single multi-FASTA. 

- <b> A priori, do you expect to achieve better or worse compression here than random data? Why? </b>
I would expect the compression here to work worse since there are often more randomized sequences than repeated ones in real DNA sequences.
 
Now, compress the multi-FASTA using gzip and bzip2. 

% Compression:
- gzip - random: 70.7, gp120: 75.4
- bzip2 - random: 72.6, gp120: 78.2
- pbzip2 - random: 72.6, gp120: 78.2


- <b>How does the compression ratio of this file compare to random data? </b>
It appears that the compression ratios for gzip, bzip, and pbzip are better in gp120 homologs FASTA file in comparison to the randomized data we generated in part 1.

<b>Estimating compression of 1000 terabytes</b>

Given the benchmarking data you obtained in this lab...

<b>Which algorithm do you propose to use for each type of data?</b>

For re-sequencing of similar genomes, I propose using pbzip2 for parallel compression.

For protein sequences, I propose using pbzip2 since it would also work fastest among the algorithms, while simultanesouly providing approximately the same compression as gzip2.

For random binary microscope images, it appears that gzip2 provides the best compression.

Provide an estimate for the fraction of space you can save using your compression scheme.

- for 800TB (of genomes and plasmids): 78.2% compressed
- for 100TB (of protein sequences): 71.2% compressed
- for 100TB (of a binary image): 0% compressed
- in total, 174.4 + 28.8 + 0 = 203.2 TB / 1000TB (79.68% compressed)

<b>How much of a bonus do you anticipate receiving this year?</b>
- Bonus = 50 dollars/TB of hard disk space saved
- 50 dollars x 796.8 TB saved = 39,840 dollars/day
- Annual bonus = 39,840 dollars/day * 365 days/year = 14,541,600 dollars for the entire year